In [1]:
# Recipe 5-1. Create a paired RDD 
# Recipe 5-2. Perform data aggregation 
# Recipe 5-3. Join data 
# Recipe 5-4. Calculate page rank

In [2]:
# Step 5-1-1. Creating an RDD with Single Elements 

pythonList  =  ['b' , 'd', 'm', 't', 'e', 'u'] 
RDD1 = sc.parallelize(pythonList,2) 
RDD1.collect()

Out[ 1 ]: ['b', 'd', 'm', 't', 'e', 'u']

In [3]:
# Step 5-1-2. Writing a Python Method to Check for Consonants 

def vowelCheckFunction(data):
  if data in ['a','e','i','o','u']:
    return 1
  else:
    return 0
  
print('a', vowelCheckFunction('a'))
print('b', vowelCheckFunction('b'))

a 1
b 0

In [4]:
# Step 5-1-3. Creating a Paired RDD

RDD2 = RDD1.map(lambda data: (data, vowelCheckFunction(data)))
RDD2.collect()

Out[ 5 ]: [('b', 0), ('d', 0), ('m', 0), ('t', 0), ('e', 1), ('u', 1)]

In [5]:
# Step 5-1-4. Fetching Keys from a Paired RDD 

RDD2Keys = RDD2.keys() 
RDD2Keys.collect()

Out[ 6 ]: ['b', 'd', 'm', 't', 'e', 'u']

In [6]:
# Step 5-1-5. Fetching Values from a Paired RDD

RDD2Values = RDD2.values()
RDD2Values.collect()

Out[ 7 ]: [0, 0, 0, 0, 1, 1]

In [7]:
# Step 5-2-1. Creating an RDD with Single Elements

filDataSingle = [['filamentA','100W',605],
                  ['filamentB','100W',683],
                  ['filamentB','100W',691], 
                  ['filamentB','200W',561],
                  ['filamentA','200W',530],
                  ['filamentA','100W',619],
                  ['filamentB','100W',686],
                  ['filamentB','200W',600],
                  ['filamentB','100W',696],
                  ['filamentA','200W',579],
                  ['filamentA','200W',520],
                  ['filamentA','100W',622],
                  ['filamentA','100W',668],
                  ['filamentB','200W',569],
                  ['filamentB','200W',555],
                  ['filamentA','200W',541]]

filDataSingleRDD = sc.parallelize(filDataSingle,2)
filDataSingleRDD.take(3)

Out[ 22 ]: 
[['filamentA', '100W', 605],
 ['filamentB', '100W', 683],
 ['filamentB', '100W', 691]]

In [8]:
# Step 5-2-2. Creating a Paired RDD 

filDataPairedRDD1 = filDataSingleRDD.map(lambda x : (x[0], x[2]))
filDataPairedRDD1.take(4)

Out[ 23 ]: 
[('filamentA', 605),
 ('filamentB', 683),
 ('filamentB', 691),
 ('filamentB', 561)]

In [9]:
# Step 5-2-3. Finding the Mean Lifetime Based on Filament Type 

filDataPairedRDD11 = filDataPairedRDD1.map(lambda x : (x[0], [x[1], 1]))
filDataPairedRDD11.take(4)

Out[ 24 ]: 
[('filamentA', [605, 1]),
 ('filamentB', [683, 1]),
 ('filamentB', [691, 1]),
 ('filamentB', [561, 1])]

In [10]:
filDataSumandCount = filDataPairedRDD11.reduceByKey(lambda l1,l2 : [l1[0] + l2[0] , l1[1]+l2[1]])
filDataSumandCount.collect()

Out[ 25 ]: [('filamentB', [5041, 8]), ('filamentA', [4684, 8])]

In [11]:
print(filDataPairedRDD11.count())
print(filDataPairedRDD11.getNumPartitions())
print(filDataPairedRDD11.take(5))
print(filDataSumandCount.collect())
filDataMeanandCount = filDataSumandCount.map( lambda l : [l[0], float(l[1][0])/l[1][1],l[1][1]]) #summation of the life hours 
filDataMeanandCount.collect()

16
2
[('filamentA', [605, 1]), ('filamentB', [683, 1]), ('filamentB', [691, 1]), ('filamentB', [561, 1]), ('filamentA', [530, 1])]
[('filamentB', [5041, 8]), ('filamentA', [4684, 8])]
 Out[ 30 ]: [['filamentB', 630.125, 8], ['filamentA', 585.5, 8]]

In [12]:
# Step 5-2-4. Finding the Mean Lifetime Based on Bulb Power 

filDataPairedRDD2 = filDataSingleRDD.map(lambda x : (x[1], x[2])) #created paired
print(filDataPairedRDD2.take(4))
fillDataPairedRDD22 = filDataPairedRDD2.map( lambda x : (x[0],[x[1],1]))
print(fillDataPairedRDD22.take(4))
powerSumandCount = fillDataPairedRDD22.reduceByKey(lambda l1,l2 : [l1[0]+l2[0], l1[1]+l2[1]]) #sum of total life hours and count
print(powerSumandCount.collect())
meanandCountPowerWise =powerSumandCount.map(lambda val : [val[0],[float(val[1][0])/val[1][1],val[1][1]]]) #mean
print(meanandCountPowerWise.collect())

[('100W', 605), ('100W', 683), ('100W', 691), ('200W', 561)]
[('100W', [605, 1]), ('100W', [683, 1]), ('100W', [691, 1]), ('200W', [561, 1])]
[('100W', [5270, 8]), ('200W', [4455, 8])]
[['100W', [658.75, 8]], ['200W', [556.875, 8]]]

In [13]:
# Step 5-2-5. Finding the Mean Lifetime Based on Filament Type and Power 

print(filDataSingleRDD.take(4))
filDataComplexKeyData = filDataSingleRDD.map( lambda val : [(val[0], val[1]),val[2]]) #combination of filament type and bulb power
print(filDataComplexKeyData.take(4))
filDataComplexKeyData1 = filDataComplexKeyData.map(lambda val : [val[0] ,[val[1],1]]) #extra 1 value
print(filDataComplexKeyData1.take(4))
filDataComplexKeySumCount = filDataComplexKeyData1.reduceByKey(lambda l1,l2 : [l1[0]+l2[0], l1[1]+l2[1]]) #get sum and count based on complex keys
print(filDataComplexKeySumCount.collect())
filDataComplexKeyMeanCount = filDataComplexKeySumCount.map(lambda val : [val[0],[float(val[1][0])/val[1][1],val[1][1]]]) #mean
print(filDataComplexKeyMeanCount.collect())

[['filamentA', '100W', 605], ['filamentB', '100W', 683], ['filamentB', '100W', 691], ['filamentB', '200W', 561]]
[[('filamentA', '100W'), 605], [('filamentB', '100W'), 683], [('filamentB', '100W'), 691], [('filamentB', '200W'), 561]]
[[('filamentA', '100W'), [605, 1]], [('filamentB', '100W'), [683, 1]], [('filamentB', '100W'), [691, 1]], [('filamentB', '200W'), [561, 1]]]
[(('filamentB', '100W'), [2756, 4]), (('filamentA', '200W'), [2170, 4]), (('filamentB', '200W'), [2285, 4]), (('filamentA', '100W'), [2514, 4])]
[[('filamentB', '100W'), [689.0, 4]], [('filamentA', '200W'), [542.5, 4]], [('filamentB', '200W'), [571.25, 4]], [('filamentA', '100W'), [628.5, 4]]]

In [14]:
# Step 5-3-1. Creating Nested Lists

studentData = [['si1','Robin','M'],
               ['si2','Maria','F'], 
               ['si3','Julie','F'],
               ['si4','Bob',  'M'], 
               ['si6','William','M']]
subjectsData = [['si1','Python'],
                ['si3','Java'],
                ['si1','Java'],
                ['si2','Python'],
                ['si3','Ruby'],
                ['si4','C++'],
                ['si5','C'],
                ['si4','Python'],
                ['si2','Java']]

In [15]:
# Step 5-3-2. Creating a Paired RDD of Students and Subjects

studentRDD = sc.parallelize(studentData, 2)
print(studentRDD.take(4))
studentPairedRDD = studentRDD.map(lambda val : (val[0],[val[1],val[2]])) #transfer to paired RDD
print(studentPairedRDD.take(4))
subjectsPairedRDD = sc.parallelize(subjectsData, 2) #paired subjects RDD 
print(subjectsPairedRDD.take(4))

[['si1', 'Robin', 'M'], ['si2', 'Maria', 'F'], ['si3', 'Julie', 'F'], ['si4', 'Bob', 'M']]
[('si1', ['Robin', 'M']), ('si2', ['Maria', 'F']), ('si3', ['Julie', 'F']), ('si4', ['Bob', 'M'])]
[['si1', 'Python'], ['si3', 'Java'], ['si1', 'Java'], ['si2', 'Python']]

In [16]:
# Step 5-3-3. Performing an Inner Join 

studenSubjectsInnerJoin = studentPairedRDD.join(subjectsPairedRDD) #common to the Students and Subjects tables
studenSubjectsInnerJoin.collect()

Out[ 43 ]: 
[('si4', (['Bob', 'M'], 'C++')),
 ('si4', (['Bob', 'M'], 'Python')),
 ('si3', (['Julie', 'F'], 'Java')),
 ('si3', (['Julie', 'F'], 'Ruby')),
 ('si2', (['Maria', 'F'], 'Python')),
 ('si2', (['Maria', 'F'], 'Java')),
 ('si1', (['Robin', 'M'], 'Python')),
 ('si1', (['Robin', 'M'], 'Java'))]

In [17]:
# Step 5-3-4. Performing a Left Outer Join

studentSubjectsleftOuterJoin = studentPairedRDD.leftOuterJoin(subjectsPairedRDD) #si6 doesn’t have counterpart inSubjects 
studentSubjectsleftOuterJoin.collect()

Out[ 46 ]: 
[('si4', (['Bob', 'M'], 'C++')),
 ('si4', (['Bob', 'M'], 'Python')),
 ('si6', (['William', 'M'], None)),
 ('si3', (['Julie', 'F'], 'Java')),
 ('si3', (['Julie', 'F'], 'Ruby')),
 ('si2', (['Maria', 'F'], 'Python')),
 ('si2', (['Maria', 'F'], 'Java')),
 ('si1', (['Robin', 'M'], 'Python')),
 ('si1', (['Robin', 'M'], 'Java'))]

In [18]:
# Step 5-3-5. Performing a Right Outer Join 

studentSubjectsrightOuterJoin = studentPairedRDD.rightOuterJoin(subjectsPairedRDD)
studentSubjectsrightOuterJoin.collect()

Out[ 48 ]: 
[('si4', (['Bob', 'M'], 'C++')),
 ('si4', (['Bob', 'M'], 'Python')),
 ('si3', (['Julie', 'F'], 'Java')),
 ('si3', (['Julie', 'F'], 'Ruby')),
 ('si5', (None, 'C')),
 ('si2', (['Maria', 'F'], 'Python')),
 ('si2', (['Maria', 'F'], 'Java')),
 ('si1', (['Robin', 'M'], 'Python')),
 ('si1', (['Robin', 'M'], 'Java'))]

In [19]:
# Step 5-3-6. Performing a Full Outer Join 

studentSubjectsfullOuterJoin = studentPairedRDD.fullOuterJoin(subjectsPairedRDD)
studentSubjectsfullOuterJoin.collect()

Out[ 49 ]: 
[('si4', (['Bob', 'M'], 'C++')),
 ('si4', (['Bob', 'M'], 'Python')),
 ('si6', (['William', 'M'], None)),
 ('si3', (['Julie', 'F'], 'Java')),
 ('si3', (['Julie', 'F'], 'Ruby')),
 ('si5', (None, 'C')),
 ('si2', (['Maria', 'F'], 'Python')),
 ('si2', (['Maria', 'F'], 'Java')),
 ('si1', (['Robin', 'M'], 'Python')),
 ('si1', (['Robin', 'M'], 'Java'))]

In [20]:
# Recipe 5-4. Calculate Page Rank 
# Contribution to a page = page rank of contributing page / total number of outbounds page from the contributing page

In [21]:
# Step 5-4-1. Creating Nested Lists of Web Pages with Outbound Links and Initializing Rank

pageLinks =  [['a', ['b','c','d']],
               ['c', ['b']],
               ['b', ['d','c']],
               ['d', ['a','c']]] #nested list of web pages and their outbound links
pageRanks =  [['a',1],
               ['c',1], 
               ['b',1], 
               ['d',1]]
#define the number of iterations for running the page rank

In [22]:
# Step 5-4-2. Writing a Function to Calculate Contributions 

def rankContribution(uris, rank):
  numberOfUris = len(uris)
  rankContribution = float(rank) / numberOfUris
  newrank = []
  for uri in uris:
    newrank.append((uri, rankContribution))
    return newrank


In [23]:
# Step 5-4-3. Creating Paired RDDs 

pageLinksRDD  = sc.parallelize(pageLinks, 2)
print(pageLinksRDD.collect())
pageRanksRDD  = sc.parallelize(pageRanks, 2)
print(pageRanksRDD.collect())

[['a', ['b', 'c', 'd']], ['c', ['b']], ['b', ['d', 'c']], ['d', ['a', 'c']]]
[['a', 1], ['c', 1], ['b', 1], ['d', 1]]

In [24]:
# Step 5-4-4. Creating a Loop for Updating Page Rank 

numIter = 20
s = 0.85
for i in range(numIter): 
  linksRank = pageLinksRDD.join(pageRanksRDD) 
  contributedRDD = linksRank.flatMap(lambda x : rankContribution(x[1] [0],x[1][1])) 
  sumRanks = contributedRDD.reduceByKey(lambda v1,v2 : v1+v2) 
  pageRanksRDD = sumRanks.map(lambda x : (x[0],(1-s)+s*x[1])) #update rank of each web page
  
pageRanksRDD.collect()

Out[ 57 ]: 
[('a', 0.2538342463798347),
 ('d', 0.2443158738349051),
 ('b', 0.2219197031409532)]